In [1]:
import random

In [2]:
# y^2 = x^3 + a*x + b

In [3]:
def add_points(P, Q, p):
    x1, y1 = P
    x2, y2 = Q
    
    if x1 == x2 and y1 == y2:
        beta = (3*x1*x2 + a) * pow(2*y1, -1, p)
    else:
        beta = (y2 - y1) * pow(x2 - x1, -1, p)
    
    x3 = (beta*beta - x1 - x2) % p
    y3 = (beta * (x1 - x3) - y1) % p
    
    is_on_curve((x3, y3), p)
        
    return x3, y3

def is_on_curve(P, p):
    x, y = P
    assert (y*y) % p == ( pow(x, 3, p) + a*x + b ) % p
    
def apply_double_and_add_method(G, k, p):
    target_point = G
    
    k_binary = bin(k)[2:] #0b1111111001
    
    for i in range(1, len(k_binary)):
        current_bit = k_binary[i: i+1]
        
        # doubling - always
        target_point = add_points(target_point, target_point, p)
        
        if current_bit == "1":
            target_point = add_points(target_point, G, p)
    
    is_on_curve(target_point, p)
    
    return target_point

In [4]:
# Secp256k1
a = 0; b = 7
G = (55066263022277343669578718895168534326250603453777594175500187360389116729240, 
     32670510020758816978083085130507043184471273380659243275938904335757337482424)

p = pow(2, 256) - pow(2, 32) - pow(2, 9) - pow(2, 8) - pow(2, 7) - pow(2, 6) - pow(2, 4) - pow(2, 0)
n = 115792089237316195423570985008687907852837564279074904382605163141518161494337

In [5]:
is_on_curve(G, p)

# Elliptic Curve ElGamal

In [6]:
# alice 
ka = random.getrandbits(256) # private of Alice
Qa = apply_double_and_add_method(G = G, k = ka, p = p)

# encryption

In [7]:
def encrypt(m, r):
    s = apply_double_and_add_method(G = G, k = m, p = p)
    
    c1 = apply_double_and_add_method(G = G, k = r, p = p) 

    c2 = apply_double_and_add_method(G = Qa, k = r, p = p)
    c2 = add_points(c2, s, p)
    
    return c1, c2

In [8]:
# bob
m = 23061912
r = random.getrandbits(128)
c1, c2 = encrypt(m, r)

In [21]:
c1, c2

((77540348118386417235088022431521225296690871290970569844578744626563260408946,
  110887220795396022362168482022447309566761338628951050256721494026565188755701),
 (50748205993558261812438974204931384216973548252261377829590065878704515903291,
  33825217600114772500241170047646282590709149318282631617258659507780877885464))

# decryption

In [9]:
# s_prime = c2 - ka x c1
# s_prime = c2 + ( ka x -c1)
# (x, y) + (x, -y) = O

In [10]:
def decrypt(c1, c2):
    c1_prime = (c1[0], (-1*c1[1]) % p)
    s_prime = apply_double_and_add_method(G = c1_prime, k = ka, p = p)
    s_prime = add_points(P = c2, Q = s_prime, p = p)
    return s_prime

In [11]:
s_prime = decrypt(c1, c2)

In [23]:
# bob actuall does not know m. he will decrypt s where s = m x G
# show s here to confirm decrypted s is equal to s itself
s_real = apply_double_and_add_method(G = G, k = m, p = p)

In [22]:
s_prime

(101401888938080707466461718009381950095311426580467620560673978804974259848061,
 81617838766714377738166229284883214383180826546122608693522719057922160918473)

In [24]:
s_real

(101401888938080707466461718009381950095311426580467620560673978804974259848061,
 81617838766714377738166229284883214383180826546122608693522719057922160918473)

In [25]:
assert s_prime == s_real

# partially homomorphic encryption

In [13]:
# (r1G, r1Q+m1G)
# (r2G, r2Q+m2G)
# (r1G + r2G, r1Q+m1G + r2Q+m2G)
# (r1 + r2)G, (r1+r2)Q + (m1+m2)G

In [14]:
m1 = 333
m2 = 777

r1 = random.getrandbits(128)
r2 = random.getrandbits(128)

In [15]:
c1_x, c2_x = encrypt(m1, r1)
c1_y, c2_y = encrypt(m2, r2)

In [16]:
m1_encrypted_plus_m2_encrypted = (
    add_points(
        P = c1_x, 
        Q = c1_y, 
        p = p
    ), 
    add_points(
        P = c2_x, 
        Q = c2_y, 
        p = p
    )
)

In [17]:
m1_plus_m2_encrypted = encrypt(m1+m2, r1+r2)

In [18]:
m1_encrypted_plus_m2_encrypted

((14865980593939876698068876225599263321251036940474617606547871128143773286814,
  46077028121493745938979093690143736399505333984612654865333216947917716654991),
 (75741602061653471892719211543889485322291173372919174930120720881734323798977,
  42804451477266150613093421250399320812634330922132137062522288638099329573521))

In [19]:
m1_plus_m2_encrypted

((14865980593939876698068876225599263321251036940474617606547871128143773286814,
  46077028121493745938979093690143736399505333984612654865333216947917716654991),
 (75741602061653471892719211543889485322291173372919174930120720881734323798977,
  42804451477266150613093421250399320812634330922132137062522288638099329573521))

In [20]:
assert m1_encrypted_plus_m2_encrypted == m1_plus_m2_encrypted